<a href="https://colab.research.google.com/github/Vijay1126/NLP-DL-Projects/blob/master/Movie_Review_Data_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/review_polarity.zip
from nltk.corpus import stopwords
import string
import re

Archive:  /content/review_polarity.zip
replace __MACOSX/._review_polarity? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace review_polarity/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
def loaddata(filename):
  file = open(filename,"r")
  text = file.read()
  file.close()
  return text

In [ ]:
from os import listdir
def loadFromDirectory(directory):
  
  for filename in listdir(directory):
    if not filename.endswith(".txt"):
      pass
    path = directory+'/'+filename
    doc = loaddata(path)
  return doc
    

In [ ]:
directory = '/content/review_polarity/txt_sentoken/neg'
loadFromDirectory(directory)

In [ ]:
filename = '/content/review_polarity/txt_sentoken/neg/cv002_17424.txt'
text = loaddata(filename)
tokens = text.split()

Before using regEx to filter the data


In [ ]:
print(tokens)

['it', 'is', 'movies', 'like', 'these', 'that', 'make', 'a', 'jaded', 'movie', 'viewer', 'thankful', 'for', 'the', 'invention', 'of', 'the', 'timex', 'indiglo', 'watch', '.', 'based', 'on', 'the', 'late', "1960's", 'television', 'show', 'by', 'the', 'same', 'name', ',', 'the', 'mod', 'squad', 'tells', 'the', 'tale', 'of', 'three', 'reformed', 'criminals', 'under', 'the', 'employ', 'of', 'the', 'police', 'to', 'go', 'undercover', '.', 'however', ',', 'things', 'go', 'wrong', 'as', 'evidence', 'gets', 'stolen', 'and', 'they', 'are', 'immediately', 'under', 'suspicion', '.', 'of', 'course', ',', 'the', 'ads', 'make', 'it', 'seem', 'like', 'so', 'much', 'more', '.', 'quick', 'cuts', ',', 'cool', 'music', ',', 'claire', "dane's", 'nice', 'hair', 'and', 'cute', 'outfits', ',', 'car', 'chases', ',', 'stuff', 'blowing', 'up', ',', 'and', 'the', 'like', '.', 'sounds', 'like', 'a', 'cool', 'movie', ',', 'does', 'it', 'not', '?', 'after', 'the', 'first', 'fifteen', 'minutes', ',', 'it', 'quickly'

After

In [ ]:

re_punc = re.compile('[%s]' % re.escape(string.punctuation))
tokens = [re_punc.sub('',w) for w in tokens]
print(tokens)

['it', 'is', 'movies', 'like', 'these', 'that', 'make', 'a', 'jaded', 'movie', 'viewer', 'thankful', 'for', 'the', 'invention', 'of', 'the', 'timex', 'indiglo', 'watch', '', 'based', 'on', 'the', 'late', '1960s', 'television', 'show', 'by', 'the', 'same', 'name', '', 'the', 'mod', 'squad', 'tells', 'the', 'tale', 'of', 'three', 'reformed', 'criminals', 'under', 'the', 'employ', 'of', 'the', 'police', 'to', 'go', 'undercover', '', 'however', '', 'things', 'go', 'wrong', 'as', 'evidence', 'gets', 'stolen', 'and', 'they', 'are', 'immediately', 'under', 'suspicion', '', 'of', 'course', '', 'the', 'ads', 'make', 'it', 'seem', 'like', 'so', 'much', 'more', '', 'quick', 'cuts', '', 'cool', 'music', '', 'claire', 'danes', 'nice', 'hair', 'and', 'cute', 'outfits', '', 'car', 'chases', '', 'stuff', 'blowing', 'up', '', 'and', 'the', 'like', '', 'sounds', 'like', 'a', 'cool', 'movie', '', 'does', 'it', 'not', '', 'after', 'the', 'first', 'fifteen', 'minutes', '', 'it', 'quickly', 'becomes', 'appa

In [ ]:

from nltk.corpus import stopwords 
import string
import re


def cleanDocument(tokens):
  tokens = text.split()
  # prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation)) # remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
  # remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()] # filter out stop words
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words] # filter out short tokens
  tokens = [word for word in tokens if len(word) > 1]
  return tokens

In [ ]:
filename = '/content/review_polarity/txt_sentoken/neg/cv005_29357.txt' 
text = loaddata(filename)
tokens = cleanDocument(text)
print(tokens)


['capsule', 'planet', 'mars', 'police', 'taking', 'custody', 'accused', 'murderer', 'face', 'title', 'menace', 'lot', 'fighting', 'whole', 'lot', 'story', 'otherwise', 'john', 'carpenter', 'reprises', 'many', 'ideas', 'previous', 'films', 'especially', 'assault', 'precinct', 'new', 'film', 'comes', 'homage', 'john', 'carpenter', 'apparently', 'believes', 'action', 'scenes', 'people', 'fight', 'something', 'horrible', 'horror', 'scenes', 'writer', 'director', 'horror', 'films', 'supposedly', 'expert', 'horror', 'bad', 'mistake', 'make', 'ghosts', 'mars', 'called', 'horror', 'movie', 'drawn', 'fight', 'humans', 'surprisingly', 'lowpowered', 'alien', 'menace', 'addition', 'anybody', 'john', 'carpenter', 'made', 'ghosts', 'mars', 'carpenter', 'would', 'grounds', 'sue', 'film', 'chock', 'full', 'pieces', 'taken', 'assault', 'precinct', 'thing', 'prince', 'darkness', 'fact', 'surprising', 'carpenter', 'managed', 'fit', 'many', 'pieces', 'previous', 'work', 'film', 'admittedly', 'novel', 'way